## Final Project Submission

Please fill out:
* Student name: 
* Student pace: self paced / part time / full time
* Scheduled project review date/time: 
* Instructor name: 
* Blog post URL:


## Conclusions:

**1.	Model Performance:** The linear regression model achieved a training score of 84.44% and a test score of 81.02%. These scores indicate that the model explains about 84.44% of the variance in the training data and performs well in predicting house prices, as evidenced by the test score.

**2.	Significant Features:** Several features have significant coefficients, indicating their importance in predicting house prices. For example, waterfront properties, excellent views, and higher grades significantly increase the house prices, while being located in certain zip codes or having poor conditions can decrease prices.

**3.	Age of the Property:** The age of the property (both built and renovated) has a significant impact on the house prices. Newer properties tend to have higher prices, while older properties may experience a decrease in value unless renovated.

**4.	Location:** Location plays a crucial role in determining house prices. Certain zip codes have significantly higher coefficients, indicating higher prices in those areas. Buyers should consider the location carefully when making a purchase decision.

## Recommendations:

**1.	Focus on Key Features:** Buyers and sellers should pay attention to key features such as waterfront views, property condition, and grade when assessing house prices. These features have significant impacts on the value of the property.

**2.	Consider Location:** Location is a critical factor in determining house prices. Buyers should consider factors such as proximity to amenities when evaluating a property. Sellers should highlight attractive aspects of the property's location in their listings.

**3.	Evaluate Property Age:** Both the age of the property and any renovations should be carefully evaluated. Buyers should inquire about the age of key components such as the roof, system, and appliances. Sellers should consider investing in renovations to increase the value of their property.

**4.	Consider number of floors:** Assess whether properties with more floors tend to command higher sale prices, controlling for other relevant factors property condition. By analyzing the relationship between the number of floors and sale prices comprehensively, we can determine whether adding floors significantly influences home prices in the King County real estate market.